# Profile Training Jobs with Amazon SageMaker Debugger

## Configuring a training job to use SageMaker Debugger

#### The first step is to configure training jobs to use Amazon SageMaker Debugger. By now, you are familiar with using the Estimator object from SageMaker SDK to launch training jobs. To use Amazon SageMaker Debugger, you must enhance Estimator with three additional configuration parameters: DebuggerHookConfig, Rules, and ProfilerConfig

#### With DebuggerHookConfig, you can specify which debugging metrics to collect and where to store them, as shown in the following code block:

In [ ]:
Estimator(
    debugger_hook_config=DebuggerHookConfig(
        s3_output_path=bucket_path,  # Where the debug data is stored.
        collection_configs=[ # Organize data to collect into collections.
            CollectionConfig(
                name="metrics",
                parameters={
                    "save_interval": str(save_interval)
                }
            )
        ],
    ),
)

#### s3_output_path is the location where all the collected data is persisted. If this location is not specified, Debugger uses the default path, s3://<output_path>/debug-output/, where <output_path> is the output path of the SageMaker training job. The CollectionConfig list allows you to organize the debug data or tensors into collections for easier analysis. A tensor represents the state of a training network at a specific time during the training process. Data is collected at intervals, as specified by save_interval, which is the number of steps in a training run.

#### How do you know which tensors to collect? SageMaker Debugger comes with a set of built-in collections to capture common training metrics such as weights, layers, and outputs. You can choose to collect all of the available tensors or a subset of them. In the preceding code sample, Debugger is gathering the metrics collection.

In [ ]:
# Use Debugger CollectionConfig to create a custom collection

collection_configs=[
        CollectionConfig(
            name="custom_collection",
            parameters={"include_regex": ".*relu |.*tanh | *weight ",})
]

#### While DebuggerHookConfig allows you to configure and save tensors, a rule analyzes the tensors that are captured during the training for specific conditions such as loss not decreasing. SageMaker Debugger supports two different types of rules: built-in and custom. SageMaker Debugger comes with a set of built-in rules in Python that can detect and report common training problems such as overfitting, underfitting, and vanishing gradients. With custom rules, you write your own rules in Python for SageMaker Debugger to evaluate against the collected tensors.

#### For example, in the following code block, Debugger collects tensors related to the metrics collection and evaluates the tensors to detect whether the training loss is reduced throughout the training process:

In [ ]:
Estimator(
rules=[
        Rule.sagemaker(
            rule_configs.loss_not_decreasing(),
            rule_parameters={
                "collection_names": "metrics",
                "num_steps": str(save_interval * 2),
            },
        ),
    ],
)

#### Finally, ProfilerConfig allows you to collect system metrics such as CPU, GPU, Memory, I/O, and framework metrics specific to the framework being used in your training job. For the system metrics, you must specify the time interval for which you want to collect metrics, while for framework metrics, you specify the starting step and the number of steps, as shown in the following code block:

In [ ]:
Estimator(
    profiler_config = ProfilerConfig(
    ## Monitoring interval in milliseconds
    system_monitor_interval_millis=500)
    ## Start collecting metrics from step 2 and collect from the next 7 steps.
    framework_profile_params=FrameworkProfile(
    start_step=2,
    num_steps=7
))
  